<a href="https://colab.research.google.com/github/RajaSuhashKesari/PySpark-Programs/blob/main/Pyspark%20Programs/Slowly%20Changing%20Dimensions/SCD_Type_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.master("local[*]").appName("Slowly Changing Dimesions Type 1").getOrCreate()

In [3]:
target_data = [
    (101, "Suhash", "Hyderabad"),
    (102, "Rahul", "Mumbai"),
    (103, "Priya", "Chennai"),
    (104, "Amit", "Delhi")
]

target_df = spark.createDataFrame(target_data, ["emp_id", "name", "city"])

In [4]:
source_data = [
    (101, "Suhash", "Bangalore"),  # Changed city
    (102, "Rahul", "Mumbai"),      # Unchanged
    (103, "Priya R", "Chennai"),   # Changed name
    (105, "Neha", "Pune")          # New record
]

source_df = spark.createDataFrame(source_data, ["emp_id", "name", "city"])

# **Implementing SCD Type 1**

### **Extracting New Records from the Source**

In [5]:
new_records = source_df.join(target_df, on = "emp_id", how= "left_anti")

### **Extracting only updated row in the Source**

In [13]:
updated_records = target_df.alias("tgt").join(source_df.alias("src"), on = "emp_id", how = "inner").filter( (col("tgt.name") != col("src.name")) | (col("tgt.city") != col("src.city"))).select("src.*")

### **Extracting only Unchanged rows in the Target**

In [14]:
unchanged_records = target_df.join(updated_records, on = "emp_id", how="left_anti")

### **Final Target table = new_records + updated_records + unchanged_records**

In [15]:
target_df = new_records.union(updated_records).union(unchanged_records)

In [16]:
target_df.show()

+------+-------+---------+
|emp_id|   name|     city|
+------+-------+---------+
|   105|   Neha|     Pune|
|   101| Suhash|Bangalore|
|   103|Priya R|  Chennai|
|   102|  Rahul|   Mumbai|
|   104|   Amit|    Delhi|
+------+-------+---------+



In [17]:
spark.stop()